In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  3 00:35:41 2021

@author: guestt

generalised script for propagating ensembles of pulses
"""
import os
from os import listdir
from felpy.utils.os_utils import mkdir_p
from felpy.model.core.beamline import Beamline
from felpy.model.core.wavefront import Wavefront
from felpy.model.tools import propagation_parameters, scale
from felpy.model.src.coherent import construct_SA1_pulse
from wpg.optical_elements import Drift, Aperture


from wpg.wpg_uti_wf import plot_intensity_map as plot_wfr
 
from multiprocessing import Pool, cpu_count
from functools import partial
from wpg import srwlib
from labwork.about import dCache
from wpg.wpg_uti_wf import calculate_fwhm
from felpy.model.beamlines.exfel_spb.methods import get_beamline_object

def load_wfr(indir):
    wfr = Wavefront()
    wfr.load_hdf5(indir)
    return wfr


In [ ]:

def run(wfr, bl, ekev, sdir = None, VERBOSE = True):
    """

    :param wfr_file: llocation of .hdf5 file
    :param scale: bool or list of scaling parameters [nx, ny, fov] for scaling 
    of input wavefield (to be updated)
    """
    if VERBOSE:
        print("loading wavefront")
        print("sdir: {}".format(sdir))
    
    if VERBOSE: 
        print("wavefront info:")
        #print(wfr.srw_info())
        print("(nx,ny,nz): ({},{},{})".format(wfr.params.Mesh.nx, wfr.params.Mesh.ny, wfr.params.Mesh.nSlices))
        print(wfr.params.Mesh.xMax-wfr.params.Mesh.xMin, wfr.params.Mesh.yMax-wfr.params.Mesh.yMin)
        print(wfr.params.wDomain)
    
    if VERBOSE:
        print("propagating wavefront")
        #print(bl)
        
    bl.propagate(wfr)
    
    plot_wfr(wfr)
    wfr.plot_phase()
    
    wfr.store_hdf5(sdir)
    if VERBOSE:
        print("wavefront propagated")
  
def scale_by_fwhm(wfr, nx = 512, ny = 512):
    """
    DEPR.
    narrow functionality for scaling a wavefront (ie the number of pixels)
    in the source plane

    :param wfr: wpg wfr strucutre
    :param isc: ideal scale
    :param ifov: ideal field of view

    :returns wfr: scaled wpg wfr structure
    """

    ix, iy = wfr.params.Mesh.nx, wfr.params.Mesh.ny
    dx, dy = wfr.params.Mesh.xMax-wfr.params.Mesh.xMin, wfr.params.Mesh.yMax-wfr.params.Mesh.yMin
    

    fx = calculate_fwhm(wfr)['fwhm_x']*2.5
    fy = calculate_fwhm(wfr)['fwhm_y']*2.5
    print(dx, fx)

    scbl = Beamline()
    scbl.append(Drift(0), propagation_parameters(1, nx/ix, 1, ny/iy))
    #scbl.append(Aperture('r','a', 800e-06, 800e-06), propagation_parameters(800e-06/dx, nx/ix, 800e-06/dx, ny/iy))

    scbl.propagate(wfr)

    return wfr

def execute():
    
    ekev = 4.96

    z = 3.644

    bl = get_beamline_object(ekev = 4.96 , crop = None, theta_HOM = 2.3e-03, theta_KB = 3e-03)
    
    sdir = dCache + "/testing_EHC/"
    mkdir_p(sdir)
    sfile = sdir + "postNVE"
    #sfile = sdir + "postslit"

    

    wfr = Wavefront()
    wfr.load_hdf5(dCache + "NanoKB-Pulse/source/NanoKB-Pulse_2.h5")
    #wfr.load_hdf5(sfile)
    wfr.set_electric_field_representation('f')
    plot_wfr(wfr)
    print("wavefront info:")
    #print(wfr.srw_info())
    print("(nx,ny,nz): ({},{},{})".format(wfr.params.Mesh.nx, wfr.params.Mesh.ny, wfr.params.Mesh.nSlices))
    print(wfr.params.Mesh.xMax-wfr.params.Mesh.xMin, wfr.params.Mesh.yMax-wfr.params.Mesh.yMin)
    print(wfr.params.wDomain)

    wfr = scale_by_fwhm(wfr, 1024, 1024)
    #run(wfr, bl, ekev = 4.96, sdir = sfile)
    
    #for i in range(100):
     #   bl = Beamline()
    
      #  bl.append(Drift(i*z/100), propagation_parameters(1,1,1,1,mode = 'fresnel'))
       # bl.propagate(wfr)
        #print((i*z)/(100*z))
    
    bl.propagate_sequential(wfr)
    plot_wfr(wfr)
    wfr.store_hdf5(sfile)
    
    return wfr

if __name__ == '__main__':
    
    
    sdir = dCache + "/testing_EHC/"
    mkdir_p(sdir)
    sfile = sdir + "postNVE"
    #sfile = sdir + "postslit"

    from multiprocessing import pool, cpu_count
    from copy import deepcopy
    from wpg.wpg_uti_wf import plot_intensity_map as pl
    from labwork.about import dCache

    wfr = execute()
    
    #for i in range(1,5):
     #   print("********************************")
      #  print(i)
       # wfr = Wavefront()
        #wfr.load_hdf5(sfile)

        #z = 3.644
        #bl = Beamline()
        #bl.append(Drift(z), propagation_parameters(1/i,1,1/i,1,mode = 'quadratic'))
        #bl.propagate(wfr)
        #plot_wfr(wfr)
        #print()
        #print("********************************")



In [ ]:
z = 2.2+3.644
from labwork.about import dCache
sdir = dCache + "/testing_EHC/"

sfile = sdir + "postNVE"

wfr = Wavefront()
wfr.load_hdf5(sfile)
bl = Beamline()
bl.append(Drift(z), propagation_parameters(1,1,1,1,mode = 'quadratic'))
bl.propagate(wfr)
plot_wfr(wfr)
wfr.store_hdf5(sdir + "PAM_truncated")
wfr.analysis(DEBUG = True)

In [ ]:
z = 2.2 
print("doing")
sdir = dCache + "/testing_EHC/"

sfile = sdir + "postNVE"

wfr = Wavefront()
wfr.load_hdf5(sfile)
bl = Beamline()
bl.append(Drift(z), propagation_parameters(5,1,5,1,mode = 'converge'))
bl.propagate(wfr)
plot_wfr(wfr)
wfr.store_hdf5(sdir + "focus")
wfr.analysis(DEBUG = True)

In [ ]:
wfr = Wavefront()
print("loading")
sdir = dCache + "/testing_EHC/"
import numpy as nphttps://www.youtube.com/watch?v=Le_2T0S8EcY
wfr.load_hdf5(sdir + "PAM")
print("loaded")
ii = wfr.get_intensity().sum(-1)
#ii = np.load(sdir + "PAM_intensity.npy")
from felpy.utils.np_utils import get_mesh
mesh = wfr.get_mesh()
#mesh = get_mesh(ii, *wfr.get_spatial_resolution())
from felpy.utils.vis_utils import basic_plot
basic_plot(ii, np.flip(mesh), cmap = 'jet')
clen = wfr.get_coherence_time()
#wfr.analysis(DEBUG = True)

In [ ]:
np.flip(mesh).shape